<a href="https://colab.research.google.com/github/JasminiSantos/Trabalho-6-TFIDF-e-Cosseno/blob/main/Tarefa_6_TFIDF_e_Cosseno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Jasmini Rebecca Gomes dos Santos**

# Extração de sentenças das urls de websites com temas relacionados à linguagem de processamento natural

In [2]:
# Jasmini Rebecca Gomes dos Santos
# Obs: Execute todas as células para conseguir rodar o código corretamente
'''
1. Sua tarefa será gerar a matriz termo-documento usando TF-IDF por meio da aplicação das
fórmulas TF-IDF na matriz termo-documento criada com a utilização do algoritmo Bag of
Words. Sobre o Corpus que recuperamos anteriormente. O entregável desta tarefa é uma
matriz termo-documento onde a primeira linha são os termos e as linhas subsequentes são
os vetores calculados com o TF-IDF.
2. Sua tarefa será gerar uma matriz de distância, computando o cosseno do ângulo entre todos
os vetores que encontramos usando o tf-idf. Para isso use a seguinte fórmula para o cálculo
do cosseno use a fórmula apresentada em Word2Vector (frankalcantara.com)
(https://frankalcantara.com/Aulas/Nlp/out/Aula4.html#/0/4/2) e apresentada na figura a
seguir:
O resultado deste trabalho será uma matriz que relaciona cada um dos vetores já calculados
com todos os outros vetores disponíveis na matriz termo-documento mostrando a distância
entre cada um destes vetores. 
'''

from __future__ import unicode_literals, print_function
from spacy.lang.en import English 
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import traceback
import sys
import unicodedata
import spacy
import math
from spacy.pipeline import EntityRuler
from spacy import displacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")

# Separa texto em sentenças
def split_in_sentences(text):
    doc = nlp(text)
    return [str(sent).strip() for sent in doc.sents]

# Remove carácteres inválidos do texto
def remove_invalid_characters(text):
  text = text.strip().split("\n")
  text_array = []
  for i in range(0, len(text) - 1):
    if text[i] != "" and re.search('[a-z0-9]', text[i]):
      text[i] = unicodedata.normalize("NFKD", text[i])
      text[i] = text[i].strip()
      text_array.append(text[i])
  text = "\n".join(text_array)
  return text

# Extrai texto a partir de uma url
def extract_text_from_url(url):
    try:
      req = Request(url, headers={'User-Agent': 'XYZ/3.0'})
      html = urlopen(req, timeout=10).read()
      soup = BeautifulSoup(html, 'lxml') 
      invalid_tags = ['[document]','noscript','meta','head', 'input','script', 'style', 'svg']
      for tag in soup.find_all(invalid_tags):
          tag.extract()
      text = soup.getText()
      text = remove_invalid_characters(text)
      text_array = split_in_sentences(text)
      text = "\n".join(text_array)
      text = remove_invalid_characters(text)
      return text
    except:
        print(traceback.format_exc())
        sys.exit(-1)

# Encontra o número de palavras de um texto
def find_number_words(text):
  words_array = re.findall(r"[\w']+", text)
  return len(words_array)

# Imprime descrição dos textos extraídos da url  
def description_of_text_from_url(url):
  text = extract_text_from_url(url)
  sentence_array = text.split("\n")
  print("***************************************")
  print("Url: " + url)
  print("Number of words: " + str(find_number_words(text)))
  print("Number of sentences: " + str(len(sentence_array)))
  print("List of sentences: ")
  print(sentence_array)
  print("***************************************")
  
url1 = "https://www.ibm.com/cloud/learn/natural-language-processing"
url2 = "https://www.dominodatalab.com/blog/natural-language-in-python-using-spacy"
url3 = "https://www.analyticsvidhya.com/blog/2020/12/understanding-text-classification-in-nlp-with-movie-review-example-example/"
url4 = "https://blogs.commons.georgetown.edu/cctp-607-spring2019/2019/02/27/natural-language-processing-google-translate/"
url5 = "https://towardsai.net/p/nlp/natural-language-processing-nlp-with-python-tutorial-for-beginners-1f54e610a1a0"
urls = [url1, url2, url3, url4, url5]

# Gera a descrição do texto extraído de cada url
for url in urls:
  description_of_text_from_url(url)

***************************************
Url: https://www.ibm.com/cloud/learn/natural-language-processing
Number of words: 1901
Number of sentences: 143
List of sentences: 
['Skip to content', 'IBM Cloud Learn Hub', 'What is Natural Language Processing?', 'Natural Language Processing (NLP)', 'By:', 'IBM Cloud Education', '2 July  2020', 'Artificial intelligence', 'What is natural language processing?', 'NLP tasks', 'NLP tools and approaches', 'NLP use cases', 'Natural language processing and IBM Watson', 'Jump to ...', 'What is natural language processing?', 'NLP tasks', 'NLP tools and approaches', 'NLP use cases', 'Natural language processing and IBM Watson', 'Natural Language Processing (NLP)', 'Natural language processing strives to build machines that understand and respond to text or voice data—and respond with text or speech of their own—in much the same way humans do.', 'What is natural language processing?', 'Natural language processing (NLP) refers to the branch of computer sci

# Extraindo vocabulário dos textos

In [3]:
contractions = {
"ain't" :"am not",
"aren't": "are not",
"can't": "cannot have",
"cause": "because",
"could've": "could have",
"couldn't": "could not",
"didn't": "did not",
"doesn't": "does not",
"doesn’t": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",    
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"it’s": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"musn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"they'd":  "they would",
"they'd've": "they would have",
"they'll": "they will",    
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": "you",
" ur ": "your",
" n ": " and "
}

# Substitui contrações nas palavras
def replace_contractions(x):
    for word in x.split():
        if word in contractions:
          x = x.replace(word, contractions[word])
    return x
    
# Extrai vocabulário dos textos
def get_vocabulary_from_texts(urls):
  vocab = []
  for url in urls:
    text = extract_text_from_url(url)
    text = replace_contractions(text)
    words = re.findall(r'\w+',text)
    for word in words:
      if word not in vocab:
        vocab.append(word)
  return vocab

print(get_vocabulary_from_texts(urls))

['Skip', 'to', 'content', 'IBM', 'Cloud', 'Learn', 'Hub', 'What', 'is', 'Natural', 'Language', 'Processing', 'NLP', 'By', 'Education', '2', 'July', '2020', 'Artificial', 'intelligence', 'natural', 'language', 'processing', 'tasks', 'tools', 'and', 'approaches', 'use', 'cases', 'Watson', 'Jump', 'strives', 'build', 'machines', 'that', 'understand', 'respond', 'text', 'or', 'voice', 'data', 'with', 'speech', 'of', 'their', 'own', 'in', 'much', 'the', 'same', 'way', 'humans', 'do', 'refers', 'branch', 'computer', 'science', 'more', 'specifically', 'artificial', 'AI', 'concerned', 'giving', 'computers', 'ability', 'spoken', 'words', 'human', 'beings', 'can', 'combines', 'computational', 'linguistics', 'rule', 'based', 'modeling', 'statistical', 'machine', 'learning', 'deep', 'models', 'Together', 'these', 'technologies', 'enable', 'process', 'form', 'its', 'full', 'meaning', 'complete', 'speaker', 'writer', 's', 'intent', 'sentiment', 'drives', 'programs', 'translate', 'from', 'one', 'anot

# Bag of Words

In [4]:
# Pega todos os textos a partir das urls
def get_all_texts(urls):
  documents = []
  for url in urls:
    text = extract_text_from_url(url)
    text = replace_contractions(text)
    words = re.findall(r'\w+',text)
    documents.append(words)
  return documents

# Imprime cada vetor da matriz. Primeira posição da matri\ é o vetor de vocabulário dos textos
def print_matrix(matrix):
  for i in range(len(matrix)):
    print(str(i) + ". " + str(matrix[i]))

# Gera o bag of words a partir do vocabulário e das listas de palavras de cada documento/texto
def generate_bag_of_words(urls):
  matrix = []
  documents = get_all_texts(urls)
  vocab = get_vocabulary_from_texts(urls)
  matrix.append(vocab)
  for document in documents:
    matrix.append(document)
  return matrix

print_matrix(generate_bag_of_words(urls))

0. ['Skip', 'to', 'content', 'IBM', 'Cloud', 'Learn', 'Hub', 'What', 'is', 'Natural', 'Language', 'Processing', 'NLP', 'By', 'Education', '2', 'July', '2020', 'Artificial', 'intelligence', 'natural', 'language', 'processing', 'tasks', 'tools', 'and', 'approaches', 'use', 'cases', 'Watson', 'Jump', 'strives', 'build', 'machines', 'that', 'understand', 'respond', 'text', 'or', 'voice', 'data', 'with', 'speech', 'of', 'their', 'own', 'in', 'much', 'the', 'same', 'way', 'humans', 'do', 'refers', 'branch', 'computer', 'science', 'more', 'specifically', 'artificial', 'AI', 'concerned', 'giving', 'computers', 'ability', 'spoken', 'words', 'human', 'beings', 'can', 'combines', 'computational', 'linguistics', 'rule', 'based', 'modeling', 'statistical', 'machine', 'learning', 'deep', 'models', 'Together', 'these', 'technologies', 'enable', 'process', 'form', 'its', 'full', 'meaning', 'complete', 'speaker', 'writer', 's', 'intent', 'sentiment', 'drives', 'programs', 'translate', 'from', 'one', 'a

In [5]:
# Encontra a frequência dos termos nos textos
def get_frequency_from_bag(matrix):
  frequency_matrix = []
  frequency_matrix.append(matrix[0])
  for i in range(1, len(matrix)):
    frequency_vector = []
    for word in matrix[0]:
      frequency_vector.append(matrix[i].count(word))
    frequency_matrix.append(frequency_vector)
  return frequency_matrix
print_matrix(get_frequency_from_bag(generate_bag_of_words(urls)))


0. ['Skip', 'to', 'content', 'IBM', 'Cloud', 'Learn', 'Hub', 'What', 'is', 'Natural', 'Language', 'Processing', 'NLP', 'By', 'Education', '2', 'July', '2020', 'Artificial', 'intelligence', 'natural', 'language', 'processing', 'tasks', 'tools', 'and', 'approaches', 'use', 'cases', 'Watson', 'Jump', 'strives', 'build', 'machines', 'that', 'understand', 'respond', 'text', 'or', 'voice', 'data', 'with', 'speech', 'of', 'their', 'own', 'in', 'much', 'the', 'same', 'way', 'humans', 'do', 'refers', 'branch', 'computer', 'science', 'more', 'specifically', 'artificial', 'AI', 'concerned', 'giving', 'computers', 'ability', 'spoken', 'words', 'human', 'beings', 'can', 'combines', 'computational', 'linguistics', 'rule', 'based', 'modeling', 'statistical', 'machine', 'learning', 'deep', 'models', 'Together', 'these', 'technologies', 'enable', 'process', 'form', 'its', 'full', 'meaning', 'complete', 'speaker', 'writer', 's', 'intent', 'sentiment', 'drives', 'programs', 'translate', 'from', 'one', 'a

# Dataframe da matriz de frequência gerada a partir da bag of words

In [8]:
import pandas as pd
import numpy as np
def generate_dictionaries_from_bag_of_words(urls):
  bag = generate_bag_of_words(urls)
  dictionaries = []
  for i in range(1, len(bag)):
    dictionary = dict.fromkeys(bag[i], 0)
    for word in bag[0]:
      dictionary[word] = bag[i].count(word)
    dictionaries.append(dictionary)
  return dictionaries
def generate_dataframe(urls):
  dictionaries = generate_dictionaries_from_bag_of_words(urls)
  df = pd.DataFrame(dictionaries)
  df = df.replace(np.nan, 0)
  df = df.apply(np.int64)
  display(df)
generate_dataframe(urls)

,Skip,to,content,IBM,Cloud,Learn,Hub,What,is,Natural,...,bounce,rate,Advertisement,customized,Others,uncategorized,Hit,enter,ESC,close
0,1,54,1,17,6,2,1,14,34,15,...,0,0,0,0,0,0,0,0,0,0
1,1,65,2,0,2,0,0,1,26,2,...,0,0,0,0,0,0,0,0,0,0
2,0,54,0,0,0,0,0,2,49,1,...,0,0,0,0,0,0,0,0,0,0
3,1,24,2,0,1,0,0,0,13,3,...,0,0,0,0,0,0,0,0,0,0
4,0,128,4,0,10,2,0,3,104,28,...,1,1,3,1,2,1,1,1,1,1


# TF

In [12]:
def get_all_tf(urls):
  dictionaries = generate_dictionaries_from_bag_of_words(urls)
  documents = get_all_texts(urls)
  tf_dictionaries = []
  for i in range(len(documents)):
    tf_dictionary = get_tf(dictionaries[i], documents[i])
    tf_dictionaries.append(tf_dictionary)
  return tf_dictionaries

def get_tf(dictionary, document):
  tf_dictionary = {}
  for word, count in dictionary.items():
      tf_dictionary[word] = count/float(len(document))
  return tf_dictionary

get_all_tf(urls)

[{'Skip': 0.0005232862375719519,
  'to': 0.0282574568288854,
  'content': 0.0005232862375719519,
  'IBM': 0.008895866038723181,
  'Cloud': 0.0031397174254317113,
  'Learn': 0.0010465724751439038,
  'Hub': 0.0005232862375719519,
  'What': 0.007326007326007326,
  'is': 0.017791732077446363,
  'Natural': 0.007849293563579277,
  'Language': 0.004186289900575615,
  'Processing': 0.0020931449502878076,
  'NLP': 0.017791732077446363,
  'By': 0.0005232862375719519,
  'Education': 0.0005232862375719519,
  '2': 0.0005232862375719519,
  'July': 0.0005232862375719519,
  '2020': 0.0005232862375719519,
  'Artificial': 0.0015698587127158557,
  'intelligence': 0.0031397174254317113,
  'natural': 0.0052328623757195184,
  'language': 0.01726844583987441,
  'processing': 0.0057561486132914706,
  'tasks': 0.004186289900575615,
  'tools': 0.003663003663003663,
  'and': 0.041339612768184195,
  'approaches': 0.0020931449502878076,
  'use': 0.0057561486132914706,
  'cases': 0.0015698587127158557,
  'Watson': 

# IDF

In [15]:
def get_all_idfs(urls):
  dictionaries = generate_dictionaries_from_bag_of_words(urls)
  idf_dictionaries = []
  for i in range(len(dictionaries)):
    idf_dictionary = get_idf(dictionaries[i])
    idf_dictionaries.append(idf_dictionary)
  return idf_dictionaries

def get_idf(dictionary):
  idf_dictionary = dict.fromkeys(dictionary[0].keys(), 0)
  for document in documents:
      for word, count in document.items():
          if count > 0:
              idf_dictionary[word] += 1
  
  for word, count in idfDict.items():
      idf_dictionary[word] = math.log10(len(documents) / float(value))
  return idf_dictionary 

get_all_idfs(urls)

AttributeError: ignored

# TFIDF

In [ ]:
def get_tfidf(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf